In [4]:
import tensorflow as tf


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
import kaggle

In [7]:
!kaggle datasets list -s 'iris'

ref                                                     title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
uciml/iris                                              Iris Species                                     4KB  2016-09-27 07:38:05         239188       2757  0.7941176        
arshid/iris-flower-dataset                              Iris Flower Dataset                            1010B  2018-03-22 15:18:06          43068        391  0.8235294        
vikrishnan/iris-dataset                                 Iris Dataset                                    999B  2017-08-03 16:00:44           3108         26  0.7647059        
therohk/ireland-historical-news                         Irish Times - Waxy-Wany News                    52MB  2021-09-25 10:5

In [8]:
!kaggle datasets download -d 'uciml/iris'

404 - Not Found
